In [12]:
# Imports
import pandas as pd
import time
import requests
import string
import numpy as np

from lxml import html
from bs4 import BeautifulSoup

In [13]:
# Setting base url for song list
url = 'http://www.traditionalmusic.co.uk/sea-shanty/0sea-shanty'

# Setting base url for song
song_url = 'http://www.traditionalmusic.co.uk/sea-shanty/'

In [14]:
# Creating empty list to store links
links_list = []

In [15]:
# Getting list of page links
page_list = requests.get('http://www.traditionalmusic.co.uk/sea-shanty/0sea-shanty.htm')
page_list.status_code

200

In [16]:
# Create soup for page links
page_list_soup = BeautifulSoup(page_list.content, 'lxml')
page_list_table = page_list_soup.find_all('a')

# Dropping non page links
del page_list_table[:3]
del page_list_table[-13:]

# Pulling out links
page_list_links = [link['href'] for link in page_list_table]

In [17]:
# Creating empty list for song links
links_list = []

In [18]:
# Generating a list of song links
for page in page_list_links:
    song_list = requests.get(f'http://www.traditionalmusic.co.uk/sea-shanty/{page}')
    song_list_soup = BeautifulSoup(song_list.content, 'lxml')
    song_table = song_list_soup.find_all('a')

    del song_table[:28]
    del song_table[-10:]
    song_links = [x['href'] for x in song_table]
    
    links_list = links_list + song_links

In [19]:
# Instantiating empty dictionary
shanties_dict = {}

In [20]:
# Loop to generate pull song lyrics 
for link in links_list:
    # Requesting song link
    song_res = requests.get(f'http://www.traditionalmusic.co.uk/sea-shanty/{link}')
    # Putting in a small sleep to limit requests on server to under 100 per minute
    time.sleep(.75)
    # Getting and formatting song title
    title = BeautifulSoup(song_res.text).title.text
    title = title.replace('Traditional Sea Songs & Sea Shanties - ', '')
    # Converting request to soup
    song_soup = BeautifulSoup(song_res.content, 'lxml')
    # Getting song lyrics
    lyrics = song_soup.find('pre').text
    # Updating dictionary
    shanties_dict.update({title : lyrics})

In [21]:
# Converting to DataFrame
traditional_music_songs = pd.DataFrame(shanties_dict.items(), columns=['title', 'lyrics'])

In [22]:
# Writing to csv for use in other scripts
traditional_music_songs.to_csv('../../data/traditional_music_songs.csv')